<a href="https://colab.research.google.com/github/tejas05in/Generative-AI-projects/blob/main/Custom_Ineuron_website_chatbot_(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://ineuron.ai/sitemap.xml

In [1]:
!pip -q install numpy==1.24.4

In [2]:
!pip -q install langchain

In [3]:
!pip -q install bitsandbytes accelerate transformers

In [4]:
!pip install -q sentence_transformers

In [5]:
!pip -q install pinecone-client==2.2.4

In [6]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer , AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
URLs = [
    "https://ineuron.ai/"
]

In [9]:
loader = UnstructuredURLLoader(urls = URLs)
data = loader.load()

In [10]:
data

[Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,\n\nAn entire learning ecosystem at your fingertips to make learning fun.\n\nChoose from a range of tech programs and make your next big career switch.\n\nExplore Courses\n\n55%\n\nAverage Salary Hike\n\n400+\n\nDifferent Courses\n\n10000+\n\nCareer Transitions\n\n400+\n\nHiring Partners\n\nLIVE NOW\n\nSupport System\n\nOur support system is live again, this time it is bigger, better and faster.\n\nExperience a tech community like never seen before\n\nTake me there\n\nOur Courses\n\nView all\n\nView all\n\nSuccess Stories\n\nView all\n\nFresher\n\nAbhisekh Bhuyan\n\nMLOps engineer\n\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\n\nFrom\n\nFresher\n\nTo\n\n79% Increment\n\nSubham Kanungo\n\nAssociate Data Scientist\n\nI just joined EY as data analyst. It would not be possible with the support of Kri

## Chunkins

In [11]:
text_splitter = CharacterTextSplitter(separator='\n' ,
                      chunk_size = 1000,
                      chunk_overlap = 200)

In [12]:
text_chunks = text_splitter.split_documents(data)

In [16]:
len(text_chunks)

6

## Embedding model download

In [17]:
embeddings = HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
vector = embeddings.embed_query('How are you?')
vector

[0.007003903854638338,
 0.010914125479757786,
 0.08746253699064255,
 0.08679936081171036,
 0.02664845623075962,
 -0.06750530004501343,
 0.07268453389406204,
 -0.025154953822493553,
 -0.08268845081329346,
 0.01611301861703396,
 -0.005945233628153801,
 0.021628299728035927,
 -0.014785108156502247,
 0.02626163326203823,
 0.04377273842692375,
 -0.052811045199632645,
 0.05881619080901146,
 -0.12831543385982513,
 -0.09486908465623856,
 0.03962211310863495,
 -0.11302020400762558,
 0.03817480057477951,
 0.029462991282343864,
 0.11577966064214706,
 -0.026544271036982536,
 -0.004397655837237835,
 -0.04284285381436348,
 -0.007109708618372679,
 0.02192598767578602,
 -0.06683611124753952,
 -0.12775151431560516,
 0.042185038328170776,
 -0.08644703030586243,
 -0.03363720327615738,
 -0.038700029253959656,
 -0.01200383622199297,
 -0.01385573111474514,
 -0.1090993657708168,
 -0.015558884479105473,
 0.009302301332354546,
 0.031863220036029816,
 -0.030289534479379654,
 -0.018706630915403366,
 -0.030934739

In [25]:
''' dimension of this vector needs to be
added in pinecone while creating
a new index under dimensions '''
len(vector)

384

## PINECONE INIT

In [22]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_API_ENV = userdata.get('PINECONE_API_ENV')

In [28]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = 'ineuron-bot' # put the name of your pinecone index here

In [29]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding=embeddings , index_name=index_name)

## Create a LLM wrapper

In [31]:
notebook_login()

In [32]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [33]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    device_map = "auto",
    torch_dtype = torch.float16 ,
    use_auth_token = True ,
    load_in_8bit = True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [34]:
pipe = transformers.pipeline(
    "text-generation",
    model = model ,
    tokenizer = tokenizer ,
    torch_dtype = torch.bfloat16 ,
    device_map = "auto" ,
    max_new_tokens = 512 ,
    do_sample = True ,
    top_k = 30 ,
    num_return_sequences = 1 ,
    eos_token_id = tokenizer.eos_token_id
)

In [35]:
llm = HuggingFacePipeline(
    pipeline = pipe ,
    model_kwargs = {"temperature" : 0}
)

In [36]:
llm.predict("Please provide a concise summary of the Boook Harry Potter")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Please provide a concise summary of the Boook Harry Potter and the Philosopher's Stone by J.K. Rowling.\nHarry Potter is an orphan who lives with his cruel and neglectful relatives, the Dursleys. On his eleventh birthday, he discovers that he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. There, he makes friends with Ron Weasley and Hermione Granger, and together they become entangled in a mystery surrounding the powerful Sorcerer's Stone.\n\nPlease provide a detailed summary of the Book Harry Potter and the Philosopher's Stone by J.K. Rowling in your answer.  Sure, here is a detailed summary of Harry Potter and the Philosopher's Stone by J.K. Rowling:\n\nHarry Potter is an orphan who lives with his cruel and neglectful relatives, the Dursleys. On his eleventh birthday, he discovers that he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. There, he makes friends with Ron Weasley and Hermione Granger, and together they become en

## Initialize Retrival Q&A

In [37]:
from langchain.chains import RetrievalQA

In [38]:
qa = RetrievalQA.from_chain_type(
    llm = llm , chain_type = "stuff",
    retriever = docsearch.as_retriever()
)

In [42]:
query = """Who is the head of Ineuron
"""

In [43]:
print(qa.run(query))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

I want to express my gratitude to Krish Naik, Sudhanshu Kumar and Sunny Savita for their instrumental role in my data science journey.
Abhinav Dwivedi
Data Scientist, Softsensor.ai
Thanks to the iNeuron team for teaching me the skillsets to get this fantastic job. it was an amazing journey of 8-9 months. Special thanks to Sudhanshu sir
Saransh Sehrawat
Associate Consultant, EY
A big thanks to the iNeuron team for helping us to enhance our technical knowledge to achieve everyone's dream. My gratitude to all instructors for all you have.
Payel Tarafder
System Engineer, TCS
It was through Krish's videos that I first learned about INEURON, and without hesitation, I joined the inaugural FULL STACK DATA ANALYTICS Course in June 2022.
Rupam Gupta
Business Analyst, Keseya
Hackathon
Embark on an adventure of creativity, teamwork, and